In [1]:
#inherented from th/Torch_TLTrainer_for_Alarm_refine_structure_name.ipynb

In [1]:
import sys;
import os;
import glob;
import math;
import numpy as np;
import glob;
import random;
import time;
import torch;
import torch.optim as optim;
import torch.nn as nn;
import json
sys.path.append(os.getcwd());
sys.path.append('../../')
sys.path.append(os.path.abspath('../../../'));
# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat

In [2]:
from SharedLibs.config_utility import *

In [3]:
#Reproducibility
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [4]:
pretrained_acdnet = '../../../th/resources/pretrained_models/acdnet_20khz_trained_model_fold4_91.00.pt'

## define TLTraining Generator Class
The Class is an python iterator class for generating data for trainer to train the model.

In [5]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples=None, labels=None, options=None, classes_dict=None):
        random.seed(42);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict = classes_dict

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));
        #return len(self.samples);

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)] #this is most important step here, randomly select two elements from two classes
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            # print(f"nClasses:{self.opt.nClasses}, type of mapdict:{type(self.mapdict)}, type of label1:{type(label1)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = self.mapdict[label1]- 1
            idx2 = self.mapdict[label2] - 1
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        print(f"batchIndex is {batchIndex}, total sounds is {len(sounds)}")
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.normalize(32768.0)]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

## ACDNetV2 define the acdnet model structure.
定義原本的ACDNetV2，for載入pretrained acdnet model.

In [6]:
class ACDNetV2(nn.Module):
    def __init__(self, input_length, n_class, sr, ch_conf=None):
        super(ACDNetV2, self).__init__();
        self.input_length = input_length;
        self.ch_config = ch_conf;

        stride1 = 2;
        stride2 = 2;
        channels = 8;
        k_size = (3, 3);
        n_frames = (sr/1000)*10; #No of frames per 10ms

        sfeb_pool_size = int(n_frames/(stride1*stride2));
        # tfeb_pool_size = (2,2);
        if self.ch_config is None:
            self.ch_config = [channels, channels*8, channels*4, channels*8, channels*8, channels*16, channels*16, channels*32, channels*32, channels*64, channels*64, n_class];
        # avg_pool_kernel_size = (1,4) if self.ch_config[1] < 64 else (2,4);
        fcn_no_of_inputs = self.ch_config[-1];
        conv1, bn1 = self.make_layers(1, self.ch_config[0], (1, 9), (1, stride1));
        conv2, bn2 = self.make_layers(self.ch_config[0], self.ch_config[1], (1, 5), (1, stride2));
        conv3, bn3 = self.make_layers(1, self.ch_config[2], k_size, padding=1);
        conv4, bn4 = self.make_layers(self.ch_config[2], self.ch_config[3], k_size, padding=1);
        conv5, bn5 = self.make_layers(self.ch_config[3], self.ch_config[4], k_size, padding=1);
        conv6, bn6 = self.make_layers(self.ch_config[4], self.ch_config[5], k_size, padding=1);
        conv7, bn7 = self.make_layers(self.ch_config[5], self.ch_config[6], k_size, padding=1);
        conv8, bn8 = self.make_layers(self.ch_config[6], self.ch_config[7], k_size, padding=1);
        conv9, bn9 = self.make_layers(self.ch_config[7], self.ch_config[8], k_size, padding=1);
        conv10, bn10 = self.make_layers(self.ch_config[8], self.ch_config[9], k_size, padding=1);
        conv11, bn11 = self.make_layers(self.ch_config[9], self.ch_config[10], k_size, padding=1);
        conv12, bn12 = self.make_layers(self.ch_config[10], self.ch_config[11], (1, 1));
        fcn = nn.Linear(fcn_no_of_inputs, n_class);
        nn.init.kaiming_normal_(fcn.weight, nonlinearity='sigmoid') # kaiming with sigoid is equivalent to lecun_normal in keras

        self.sfeb = nn.Sequential(
            #Start: Filter bank
            conv1, bn1, nn.ReLU(),\
            conv2, bn2, nn.ReLU(),\
            nn.MaxPool2d(kernel_size=(1, sfeb_pool_size))
        );

        tfeb_modules = [];
        self.tfeb_width = int(((self.input_length / sr)*1000)/10); # 10ms frames of audio length in seconds
        tfeb_pool_sizes = self.get_tfeb_pool_sizes(self.ch_config[1], self.tfeb_width);
        p_index = 0;
        for i in [3,4,6,8,10]:
            tfeb_modules.extend([eval('conv{}'.format(i)), eval('bn{}'.format(i)), nn.ReLU()]);

            if i != 3:
                tfeb_modules.extend([eval('conv{}'.format(i+1)), eval('bn{}'.format(i+1)), nn.ReLU()]);

            h, w = tfeb_pool_sizes[p_index];
            if h>1 or w>1:
                tfeb_modules.append(nn.MaxPool2d(kernel_size = (h,w)));
            p_index += 1;

        tfeb_modules.append(nn.Dropout(0.2));
        tfeb_modules.extend([conv12, bn12, nn.ReLU()]);
        h, w = tfeb_pool_sizes[-1];
        if h>1 or w>1:
            tfeb_modules.append(nn.AvgPool2d(kernel_size = (h,w)));
        tfeb_modules.extend([nn.Flatten(), fcn]);

        self.tfeb = nn.Sequential(*tfeb_modules);

        self.output = nn.Sequential(
            nn.Softmax(dim=1)
        );
        

    def forward(self, x):
        x = self.sfeb(x);
        #swapaxes
        x = x.permute((0, 2, 1, 3));
        x = self.tfeb(x);
        y = self.output[0](x);
        return y;

    def make_layers(self, in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
        bn = nn.BatchNorm2d(out_channels);
        return conv, bn;

    def get_tfeb_pool_sizes(self, con2_ch, width):
        h = self.get_tfeb_pool_size_component(con2_ch);
        w = self.get_tfeb_pool_size_component(width);
        # print(w);
        pool_size = [];
        for  (h1, w1) in zip(h, w):
            pool_size.append((h1, w1));
        return pool_size;

    def get_tfeb_pool_size_component(self, length):
        # print(length);
        c = [];
        index = 1;
        while index <= 6:
            if length >= 2:
                if index == 6:
                    c.append(length);
                else:
                    c.append(2);
                    length = length // 2;
            else:
               c.append(1);

            index += 1;

        return c;

def GetACDNetModel(input_len=30225, nclass=50, sr=20000, channel_config=None):
    net = ACDNetV2(input_len, nclass, sr, ch_conf=channel_config);
    return net;

## load pretrained acdnet weights of 20khz

In [6]:
# acdnet_model = GetACDNetModel()
# pretrain_weight= torch.load('./resources/pretrained_models/acdnet_20khz_trained_model_fold4_91.00.pt', map_location=torch.device('cpu'))['weight']

# model_state = acdnet_model.state_dict()
# model_state.update(pretrain_weight)
# acdnet_model.load_state_dict(pretrain_weight, strict=False)

# for k, v in pretrain_weight['weight'].items():
#     print("name:", k)
#     print("\n")

# remove the unexpected keys: weight and config
# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in checkpoint.items():
#     name = k.replace("weight", "") # remove `module.`
#     new_state_dict[name] = v
#     name = k.replace("config", "") # remove `module.`
#     new_state_dict[name] = v

# model_state = acdnet_model.state_dict()
# model_state.update(new_state_dict)
# acdnet_model.load_state_dict(new_state_dict, strict=False)

# print("acdnet_model state_dict:\n",acdnet_model.state_dict())
# print("pretrain_weight: \n",pretrain_weight)  

In [7]:
# layer_38_of_tfeb = list(acdnet_model.tfeb.children())[38]

# print(layer_38_of_tfeb)
# print(nn.Sequential(*list(acdnet_model.tfeb.children())[:-6]))
# print(nn.Sequential(*list(acdnet_model.tfeb.children())))
# print(acdnet_model)
# for item_v in nn.Sequential(*list(acdnet_model.tfeb.children())):
#     for internal_k, internal_v in item_v.named_parameters():
#         print(internal_v.requires_grad)

In [8]:
# print(acdnet_model.fc)
#acdnet 包含三部份：sfeb, tfeb and output
# print(nn.Sequential(*list(acdnet_model.children())))
# print(nn.Sequential(*list(acdnet_model.children())[:-1]))
# for k, v in acdnet_model.named_parameters():
#     print("key:", k)
#     v.requires_grad = False

# acdnet_model.fcn = nn.Linear(num_ftrs, 10)
# print(acdnet_model)

In [7]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNet_TL_Model_Extend',  required=False);
    parser.add_argument('--data', default='../datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();
    
    #Leqarning settings
    opt.batchSize = 64;
    opt.LR = 0.1;
    opt.weightDecay = 5e-2#9e-3;#5e-3;#5e-2;#1e-2;#5e-4;
    opt.momentum = 0.09;
    opt.nEpochs = 1000;
    opt.schedule = [0.3, 0.6, 0.9];
    opt.warmup = 10;
    if torch.backends.mps.is_available():
        opt.device="mps"; #for apple m2 gpu
    elif torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = 2#50;
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 2;
    opt.TLAcdnetConfig = [8,64,32,64,64,128,128,256,256,512,512,2];
    return opt
    # opt = parser.parse_args();

In [8]:
def make_layers(in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
        bn = nn.BatchNorm2d(out_channels);
        return conv, bn;

In [9]:
ch_confing_10 = 8 * 64
ch_n_class = 3
fcn_no_of_inputs = 3
# conv12, bn12 = self.make_layers(self.ch_config[10], self.ch_config[11], (1, 1));
conv12, bn12 = make_layers(in_channels = ch_confing_10, out_channels = ch_n_class, kernel_size = (1, 1));
fcn = nn.Linear(fcn_no_of_inputs, ch_n_class);

In [10]:
"""
[8,64,32,64,64,128,128,256,256,512,512, 50]
[1, 2, 3, 4, 5, 6,  7,  8,  9   10, 11, 12]
[8,64,32,64,64,128,128,256,256,512,2,2]
"""

'\n[8,64,32,64,64,128,128,256,256,512,512, 50]\n[1, 2, 3, 4, 5, 6,  7,  8,  9   10, 11, 12]\n[8,64,32,64,64,128,128,256,256,512,2,2]\n'

In [11]:
class ACDNet_TL_Model_Extend(nn.Module):
    def __init__(self, PretrainedWeights='acdnet_20khz_trained_model_fold4_91.00.pt',opt=None):
        super(ACDNet_TL_Model_Extend, self).__init__()
        acdnet_model = GetACDNetModel(); # load original acdnet model first
        # device = opt#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.opt = opt;
        self.ch_config = None;
        print(f"device is {self.opt.device}")
        pretrain_weight= torch.load(PretrainedWeights, map_location=torch.device(self.opt.device))['weight']
        model_state = acdnet_model.state_dict()
        model_state.update(pretrain_weight)
        acdnet_model.load_state_dict(pretrain_weight, strict=False)
        if self.ch_config is None:
            self.ch_config = opt.TLAcdnetConfig;
            #[channels, channels*8, channels*4, channels*8, channels*8, channels*16, channels*16, channels*32, channels*32, channels*64, channels*64, n_class];
        # print(type(acdnet_model))
        # count = 0;
        for k, v in acdnet_model.named_parameters():
            # count += 1;
            # print(f"set {k} required_grade to False");
            v.requires_grad = False
        # print(f"count is {count}");
        self.sfeb = nn.Sequential(*list(acdnet_model.children())[0])
        tfeb_modules = []
        tfeb_modules.extend([*list(acdnet_model.tfeb.children())[:-6]])
        tfeb_modules.extend([conv12, bn12, nn.ReLU()]);
        tfeb_modules.append(nn.AvgPool2d(kernel_size = (2,4)));
        tfeb_modules.extend([nn.Flatten(), fcn]);
        # self.retrained_layers = nn.Sequential(*list(acdnet_model.tfeb.children())[:-1])
        # fcn_no_of_inputs = 50, n_class=10
        # n_class=6
        # fc = nn.Linear(50, n_class);
        # fc.requires_grad = True
        # tfeb_modules.extend([fc])
        self.tfeb = nn.Sequential(*tfeb_modules)
        self.output = nn.Sequential(
        nn.Softmax(dim=1));
        # print(f"type of self.tfeb is {type(self.tfeb)}")
        # for k2, v2 in self.tfeb:
        #     print(f"k:{k}'s requires_grad is {v2.requires_grad}");

    def forward(self, x):
        # print(f"sfeb:\n{list(self.sfeb.children())}");
        print(f"input x shape:{x.size()}")
        x = self.sfeb(x);
        #swapaxes
        x = x.permute((0, 2, 1, 3));
        x = self.tfeb(x);
        y = self.output[0](x);
        return y;

In [12]:
def GetTLACDNet(opt=None):
    model = ACDNet_TL_Model_Extend(PretrainedWeights=pretrained_acdnet, opt=opt);#ACDNet_TL_Model()
    return model

In [13]:
# test_model = GetTLACDNet()
# calc.summary(test_model, (1,1,30225))
# print(test_model)
# print(test_model.state_dict())

In [14]:
from datetime import datetime;

In [15]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [16]:
"""
報錯訊息：
TypeError: can't convert np.ndarray of type numpy.object_. 
The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.
1
2
問題描述：
當把np轉換成torch tensor時，

trainx = torch.from_numpy(np.reshape(train_x, newshape=(-1,25)))
1
解決方法：
由於讀入的numpy陣列裡的元素是object類型，無法將此型別轉換成tensor。

所以，將numpy數組進行強制型別轉換成float型別（或任何pytorch支援的型別：float64, float32, float16, int64, int32, int16, int8, uint8, and bool）即可。

trainx = trainx.astype(float)  # numpy強制轉型
————————————————

"""

"\n報錯訊息：\nTypeError: can't convert np.ndarray of type numpy.object_. \nThe only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.\n1\n2\n問題描述：\n當把np轉換成torch tensor時，\n\ntrainx = torch.from_numpy(np.reshape(train_x, newshape=(-1,25)))\n1\n解決方法：\n由於讀入的numpy陣列裡的元素是object類型，無法將此型別轉換成tensor。\n\n所以，將numpy數組進行強制型別轉換成float型別（或任何pytorch支援的型別：float64, float32, float16, int64, int32, int16, int8, uint8, and bool）即可。\n\ntrainx = trainx.astype(float)  # numpy強制轉型\n————————————————\n\n"

In [17]:
"""
解決RuntimeError: Input type and weight type should be the same


根據報錯資訊的意思可以推斷，這個錯誤是由輸入和權重的資料類型不一致引起的。因此解決方法很簡單，就是將輸入的資料和模型參數的資料類型統一即可。在這個例子中，有以下幾個解決方法。

1.將輸入資料（torch.tensor 形式）轉換成FloatTensor形式，如下：

# net_in是torch.tensor形式的输入数据
net_in = net_in.float();
1
2
2.如果輸入資料在轉變為torch.tensor前是以numpy數組的形式儲存的，我們可以將資料提前轉變為float32形式，具體如下：

# train_set是numpy.array形式的输入数据
import numpy as np
X = train_set.astype(np.float32);
1
2
3
3.將模型參數類型轉換為與輸入張量（tensor）一致的型別。在這個例子裡，模型參數需轉換為DoubleTensor，如下所示：

model.double()
1
可選擇以上任一方法解決這個問題。但在實際應用上需要注意，第三種解決方法會增加顯存的需求量。更多關於torch中張量（tensor）資料類型的介紹，可參考這個網頁Link。
————————————————
版权声明：本文为CSDN博主「Henry积少成多」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_34612816/article/details/123372456
"""

'\n解決RuntimeError: Input type and weight type should be the same\n\n\n根據報錯資訊的意思可以推斷，這個錯誤是由輸入和權重的資料類型不一致引起的。因此解決方法很簡單，就是將輸入的資料和模型參數的資料類型統一即可。在這個例子中，有以下幾個解決方法。\n\n1.將輸入資料（torch.tensor 形式）轉換成FloatTensor形式，如下：\n\n# net_in是torch.tensor形式的输入数据\nnet_in = net_in.float();\n1\n2\n2.如果輸入資料在轉變為torch.tensor前是以numpy數組的形式儲存的，我們可以將資料提前轉變為float32形式，具體如下：\n\n# train_set是numpy.array形式的输入数据\nimport numpy as np\nX = train_set.astype(np.float32);\n1\n2\n3\n3.將模型參數類型轉換為與輸入張量（tensor）一致的型別。在這個例子裡，模型參數需轉換為DoubleTensor，如下所示：\n\nmodel.double()\n1\n可選擇以上任一方法解決這個問題。但在實際應用上需要注意，第三種解決方法會增加顯存的需求量。更多關於torch中張量（tensor）資料類型的介紹，可參考這個網頁Link。\n————————————————\n版权声明：本文为CSDN博主「Henry积少成多」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。\n原文链接：https://blog.csdn.net/qq_34612816/article/details/123372456\n'

In [18]:
class TLTrainer:
    def __init__(self, opt=None, classes_dict=None):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;
        self.trainGen = getTrainGen(opt,classes_dict=classes_dict)#train_generator.setup(opt, split);
        # self.opt.trainer = self;
        # self.trainGen = getTrainGen(self.opt, self.opt.splits)#train_generator.setup(self.opt, self.opt.split);
        # self.pretrainedmodelpath = "./resources/pretrained_models/acdnet20_20khz_fold4.h5"

    def Train(self):
        train_start_time = time.time();
        net = GetTLACDNet(self.opt).to(self.opt.device)#models.GetACDNetModel().to(self.opt.device);
        #print networks parameters' require_grade value
        for k_, v_ in net.named_parameters():
            print(f"{k_}:{v_.requires_grad}")
        print('ACDNet model has been prepared for training');

        calc.summary(net, (1,1,self.opt.inputLength));

        # training_text = "Re-Training" if self.opt.retrain else "Training from Scratch";
        # print("{} has been started. You will see update after finishing every training epoch and validation".format(training_text));

        lossFunc = torch.nn.KLDivLoss(reduction='batchmean');
        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True);

        # self.opt.nEpochs = 1957 if self.opt.split == 4 else 2000;
        for epochIdx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epochIdx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            for batchIdx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batchIdx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);
                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                outputs = net(x);
                running_acc += (((outputs.data.argmax(dim=1) == y.argmax(dim=1))*1).float().mean()).item();
                loss = lossFunc(outputs.log(), y);
                loss.backward();
                optimizer.step();

                running_loss += loss.item();

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;

            net.eval();
            val_acc, val_loss = self.__validate(net, lossFunc);
            #Save best model
            self.__save_model(val_acc, epochIdx, net);
            self.__on_epoch_end(epoch_start_time, epoch_train_time, epochIdx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - train_start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));

    def load_test_data(self):
        # data = np.load(os.path.join(self.opt.data, self.opt.dataset, 'test_data_{}khz/fold{}_test4000.npz'.format(self.opt.sr//1000, self.opt.split)), allow_pickle=True);
        data = np.load(self.opt.testData, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['y'])}")
        # self.testX = torch.tensor(np.moveaxis(data['x'], 3, 1)).to(self.opt.device);
        dataX = np.moveaxis(data['x'], 3, 1).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(data['y']).type(torch.float32).to(self.opt.device);

    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __get_batch(self, index):
        x = self.trainX[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        y = self.trainY[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        return x.to(self.opt.device), y.to(self.opt.device);

    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data();
        net.eval();
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#             for idx in range(len(self.testX)):
#             x = self.testX[idx*batch_size : (idx+1)*batch_size];
            x = self.testX[:];
            scores = net(x);
            y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
            acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
#         with torch.no_grad():
#             y_pred = None;
#             batch_size = (self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
#             for idx in range(math.ceil(len(self.testX)/batch_size)):
#                 x = self.testX[idx*batch_size : (idx+1)*batch_size];
#                 scores = net(x);
#                 y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));

#             acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
        net.train();
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        print(f"shape of y_pred:{y_pred.shape}");
        print(f"shape of y_target:{y_target.shape}");
        
        with torch.no_grad():
            #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
            if self.opt.nCrops == 1:
                y_pred = y_pred.argmax(dim=1);
                y_target = y_target.argmax(dim=1);
            else:
                y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
                y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
                print(f"after: len of y_pred:{len(y_pred)}, len of y_target:{len(y_target)}")
            acc = (((y_pred==y_target)*1).float().mean()*100).item();
            # valLossFunc = torch.nn.KLDivLoss();
            loss = lossFunc(y_pred.float().log(), y_target.float()).item();
            # loss = 0.0;
        return acc, loss;

    def __on_epoch_end(self, start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - start_time;
        val_time = epoch_time - train_time;
        line = 'SP-{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.splits, epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();

    def __save_model(self, acc, epochIdx, net):
        print("__save_model is called")
        print(f"current best Acc is {self.bestAcc}")
        print(f"pass in acc is {acc}")
        if acc > self.bestAcc:
            dir = os.getcwd();
            save_path = "./trained_models/{}".format(self.opt.model_name.format(genDataTimeStr(),acc,epochIdx));
            # fname = "{}/torch/trained_models/{}_fold{}.pt";
            # fname = "{}/trained_models/acdnet_torch_20231218.pt";
            # old_model = fname.format(dir, self.opt.model_name.lower(), self.opt.splits);
            # if os.path.isfile(old_model):
            #     os.remove(old_model);
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            # torch.save({'weight':net.state_dict(), 'config':net.ch_config}, fname.format(dir, self.opt.model_name.lower(), self.opt.split));
            # torch.save({'weight':net.state_dict()}, fname.format(dir, self.opt.model_name.lower(), self.opt.splits));
            torch.save({'weight':net.state_dict(), 'config':net.ch_config}, save_path);
            print(f"model saved....., acc: {acc}")

In [23]:
def getTrainGen(opt=None, fold=None, classes_dict=None):
    dataset = np.load(opt.trainData, allow_pickle=True);
    # train_sounds = [dataset['x'][i][0] for i in range(len(dataset['x']))]
    # train_labels = [dataset['y'][i][0] for i in range(len(dataset['y']))]
    train_sounds = dataset['fold1'].item()['sounds']
    train_labels = dataset['fold1'].item()['labels']
    trainGen = TLGenerator(train_sounds, train_labels, opt, classes_dict=classes_dict);
    return trainGen

In [24]:
# print(train_labels)

In [25]:
Training_Settings={
    "last_checkpoint":"../trained_models/",
    "last_best_acc":90,
    "last_acc_epochs":100,
    "last_acc_lr":0.005,
    "last_acc_weight_decay":[0.3,0.6,0.9]
}

In [28]:
def main():
    map_dict_train = {
        '52':1, #alarm
        '56':2,
        '99':3, #other_sounds
    };
    opt = getOpts();
    opt.sr = 20000;
    opt.inputLength = 30225;
    opt.trainer = None
    opt.trainData="../../../datasets/CurrentUse/generated_datasets/train/single_fold_train_20240329133216.npz";
    opt.testData="../../../datasets/CurrentUse/generated_datasets/validation/final_single_val_20240329171721.npz";
    # import torch;
    # print(f"use {opt.device} to train.")
    tlopts.display_info(opt)
    opt.model_name = "acdnet_alarm_3rd_{}_acc_{}_{}th_epoch.pt"
    # valid_path = False;
    print("Initializing TLTrainer Object.....")
    trainer = TLTrainer(opt,classes_dict=map_dict_train)
    print("Start to training.....")
    trainer.Train();

In [29]:
main()

***Use device:cuda:0
+------------------------------+
| ACDNet_TL_Model_Extend Sound classification
+------------------------------+
| dataset  : uec_iot
| nEpochs  : 1000
| LRInit   : 0.1
| schedule : [0.3, 0.6, 0.9]
| warmup   : 10
| batchSize: 64
| nFolds: 1
| Splits: [1]
+------------------------------+
Initializing TLTrainer Object.....
length of samples:356
Start to training.....
device is cuda:0
sfeb.0.weight:False
sfeb.1.weight:False
sfeb.1.bias:False
sfeb.3.weight:False
sfeb.4.weight:False
sfeb.4.bias:False
tfeb.0.weight:False
tfeb.1.weight:False
tfeb.1.bias:False
tfeb.4.weight:False
tfeb.5.weight:False
tfeb.5.bias:False
tfeb.7.weight:False
tfeb.8.weight:False
tfeb.8.bias:False
tfeb.11.weight:False
tfeb.12.weight:False
tfeb.12.bias:False
tfeb.14.weight:False
tfeb.15.weight:False
tfeb.15.bias:False
tfeb.18.weight:False
tfeb.19.weight:False
tfeb.19.bias:False
tfeb.21.weight:False
tfeb.22.weight:False
tfeb.22.bias:False
tfeb.25.weight:False
tfeb.26.weight:False
tfeb.26.bias:False

KeyError: 99

In [4]:
# acdnet original torch Train
# def Train(opt):
#     print('Starting {} model Training for Fold-{}'.format(opt.model_name.upper(), opt.split));
#     opts.display_info(opt);
#     trainer = Trainer(opt);
#     trainer.Train();

In [5]:
"""
if __name__ == '__main__':
    opt = opts.parse();
    opt.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");
    valid_training_type = False;
    while not valid_training_type:
        train_type = input('Enter an option: \n1. Re-Training\n2. Training from Scratch\n:');
        if train_type in ['1', '2']:
            opt.retrain = True if train_type == '1' else False;
            valid_training_type = True;

    valid_path = False;
    while not valid_path:
        model_path = input("Enter your pruned model path OR keep it blank to train the base ACDNet model\n:");
        opt.model_path = "ACDNet" if model_path == '' else model_path;
        if model_path == '':
            opt.model_path = "ACDNet";
            print('ACDNet base model will be trained.');
            valid_path = True;

        else:
            file_paths = glob.glob(os.path.join(os.getcwd(), model_path));
            if len(file_paths)>0 and os.path.isfile(file_paths[0]):
                state = torch.load(file_paths[0], map_location=opt.device);
                opt.model_path = file_paths[0];
                print('Model has been found at: {}'.format(opt.model_path));
                valid_path = True;

    valid_model_name = False;
    while not valid_model_name:
        model_name = input('Enter a name that will be used to save the trained model: ');
        if model_name != '':
            opt.model_name = model_name;
            valid_model_name = True;

    valid_fold = False;
    split = None;
    while not valid_fold:
        fold = input("Which fold do you want your model to be Validated:\n 0. 5-Fold Cross Validation\n 1. Fold-1\n 2. Fold-2\n 3. Fold-3\n 4. Fold-4\n 5. Fold-5\n :")
        if fold in ['0','1','2','3','4','5']:
            split = int(fold);
            valid_fold = True;

    if split == 0:
        # -- Run for all splits
        for split in opt.splits:
            opt.split = split;
            Train(opt);
    else:
        opt.split = split;
        Train(opt);
"""

'\nif __name__ == \'__main__\':\n    opt = opts.parse();\n    opt.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");\n    valid_training_type = False;\n    while not valid_training_type:\n        train_type = input(\'Enter an option: \n1. Re-Training\n2. Training from Scratch\n:\');\n        if train_type in [\'1\', \'2\']:\n            opt.retrain = True if train_type == \'1\' else False;\n            valid_training_type = True;\n\n    valid_path = False;\n    while not valid_path:\n        model_path = input("Enter your pruned model path OR keep it blank to train the base ACDNet model\n:");\n        opt.model_path = "ACDNet" if model_path == \'\' else model_path;\n        if model_path == \'\':\n            opt.model_path = "ACDNet";\n            print(\'ACDNet base model will be trained.\');\n            valid_path = True;\n\n        else:\n            file_paths = glob.glob(os.path.join(os.getcwd(), model_path));\n            if len(file_paths)>0 and os.path.i

In [15]:
settings = {
    "pt_name":"",
    "accuracy":90,
    "epochs":100,
    "lr":0.005,
    "weight_decay":[0.3,0.6,0.9]
}
json_file = "./test/test.json"

In [16]:
writeAssumeConfig(cfg_file=json_file, settings=settings)

In [18]:
read_ret = readAssumeConfig(cfg_file=json_file)
print(read_ret)

{'pt_name': '', 'accuracy': 90, 'epochs': 100, 'lr': 0.005, 'weight_decay': [0.3, 0.6, 0.9]}


In [22]:
settings = {
    "pt_name":"here",
    "accuracy":95.5,
    "epochs":100,
    "lr":0.0005,
    "weight_decay":[0.3,0.6,0.9]
}

In [23]:
writeAssumeConfig(cfg_file=json_file, settings=settings)

In [24]:
read_ret = readAssumeConfig(cfg_file=json_file)
print(read_ret)

{'pt_name': 'here', 'accuracy': 95.5, 'epochs': 100, 'lr': 0.0005, 'weight_decay': [0.3, 0.6, 0.9]}
